## Prompt Engineering

#### 1. Define Model

In [1]:
from openai import AzureOpenAI
import os

base_url = "https://aoai719xyz01.openai.azure.com/"
api_version = "2023-07-01-preview"
api_key = os.environ.get("OPENAI_API_KEY")
deployment_name = "gpt35turbo-16k" # change to your deployment name

client = AzureOpenAI(api_key=api_key, api_version=api_version, azure_endpoint=base_url)

# call OpenAI API with model name and prompt
def call_openai_api(prompt, deployment_name=deployment_name, max_token=400, stop=None, n=1, temperature=0):
    response = client.chat.completions.create(
    model=deployment_name,
    messages = [{"role":"system", "content":"You are a helpful assistant. "},
                {"role":"user","content": prompt,}],
        max_tokens=max_token,
        stop=stop,
        n=n,
        temperature=temperature,)

    return response

#### 2. Chain of Thought

Experiment results demonstrate Zero-shot-CoT using single prompt template, significantly outperform zero-shot LLM performance on diverse benchmark reasoning tasks. Without any hand-crafted few-shot examples, e.g. increasing the accuracy on MultiArith from 17.7% to 78.7% and GSM8K from 10.4% to 40.7% with large InstructGPT model (text-davinci-002).

Source: [Large Language Models are Zero-Shot Reasoners](https://arxiv.org/abs/2205.11916)

##### Recent GPT-35-Turbo has improved logic reasoning:

In [2]:
# This prompt gets wrong answer

PROMPT_ZERO_SHOT = """Q: A juggler can juggle 16 balls. Half of the balls are golf balls,
and half of the golf balls are blue. How many blue golf balls are
there?
A: The answer (arabic numerals) is
"""
response = call_openai_api(PROMPT_ZERO_SHOT)

print(response.choices[0].message.content)

4.


##### However, it still makes mistakes with one-shot learning

In [3]:
# Still wrong answer with few-shot learning

PROMPT_FEW_SHOT = """Q: Roger has 5 tennis balss. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does Roger have now?
A: The answer is 11.

Q: A juggler can juggle 16 balls. Half of the balls are golf balls and half of the golf balls are blue. How many blue golf balls are there?
A:
"""
response = call_openai_api(PROMPT_FEW_SHOT)
print(response.choices[0].message.content)

There are 8 blue golf balls.


In [4]:
# With CoT, the answer is correct

PROMPT_ZERO_SHOT_CoT = """Q: A juggler can juggle 16 balls. Half of the balls are golf balls,
and half of the golf balls are blue. How many blue golf balls are
there?
A: Let’s think step by step.
"""
response = call_openai_api(PROMPT_ZERO_SHOT_CoT)
print(response.choices[0].message.content)

First, we need to find out how many golf balls there are in total. Since half of the balls are golf balls, we can divide the total number of balls by 2. So, 16 balls divided by 2 equals 8 golf balls.

Next, we need to find out how many of these golf balls are blue. Since half of the golf balls are blue, we can again divide the number of golf balls by 2. So, 8 golf balls divided by 2 equals 4 blue golf balls.

Therefore, there are 4 blue golf balls.


In [5]:
PROMPT_FEW_SHOT_CoT = """Q: Roger has 5 tennis balls. He buys 2 more cans of tennis
balls. Each can has 3 tennis balls. How many tennis balls does
he have now?
A: Roger started with 5 balls. 2 cans of 3 tennis balls each is 6
tennis balls. 5 + 6 = 11. The answer is 11.
Q: A juggler can juggle 16 balls. Half of the balls are golf balls,
and half of the golf balls are blue. How many blue golf balls are
there?
A:
"""
response = call_openai_api(PROMPT_FEW_SHOT_CoT)
print(response.choices[0].message.content)

If half of the balls are golf balls, then there are 16/2 = 8 golf balls.
If half of the golf balls are blue, then there are 8/2 = 4 blue golf balls.
Therefore, there are 4 blue golf balls.


#### 

##### Commonsense Reasoning

Paper: [Generated Knowledge Prompting for Commonsense Reasoning](https://arxiv.org/abs/2110.08387)

Provide knowledge, turn knowledge question into reasoning. In general, more knowledge, better result.

3 Contributing factors:

(i) the quality of knowledge,

(ii) the quantity of knowledge where the performance improves with more knowledge statements, and

(iii) the strategy for integrating knowledge during inference

In [8]:
PROMPT = """The player with the lowest score wins.
Is this true or false: Part of golf is trying to get a higher point total than others.
"""
response = call_openai_api(PROMPT)
print(response.choices[0].message.content)

False. In golf, the objective is to have the lowest score possible. The player with the lowest score is the winner.


In [9]:
PROMPT = """A tripod is a kind of easel
How many legs does an easel have?
"""
response = call_openai_api(PROMPT)
print(response.choices[0].message.content)

An easel typically has three legs.
